In [1]:
#importing required libraries
import cv2
import speech_recognition as sr
import datetime
import numpy as np
import random

#defining the detect faces function
def detect_faces(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    return faces

# Define the filter functions
def apply_grayscale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

def apply_red_filter(frame):
    red_overlay = np.full_like(frame, (0, 0, 255), dtype=np.uint8)  # Create a red overlay
    return cv2.addWeighted(frame, 1.0, red_overlay, 0.5, 0)  # Blend the overlay with the frame

# Define the available filters
FILTERS = {
    "grayscale": apply_grayscale,
    "red": apply_red_filter,
}
#specifing the image path
image_path = 'C:/Users/nomaa/OneDrive/Desktop/nomaan1.jpg'

#defining the recognize uploaded image function

def recognize_uploaded_image(image_path, face_cascade, frame):#passing image path and face cascade and frame variables
    faces = detect_faces(frame, face_cascade)

    if len(faces) == 0:
        cv2.putText(frame, "Face not detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    else:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, "Image has been Recognized", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Resize the frame to a smaller size
    resized_frame = cv2.resize(frame, (300, 300))  # Adjust the size as needed
    
    cv2.imshow('Face Detection', resized_frame)
    cv2.waitKey(2000)
    
#defining play number game function
def play_number_guessing_game(r, frame):
    print("Welcome to the Number Guessing Game!")
    secret_number = random.randint(1, 100)
    attempts = 0
    max_attempts = 5
    
    #handling attempts
    while attempts < max_attempts:
        try:
            with sr.Microphone() as source:
                print("Please say your guess (between 1 and 100):")
                audio_text = r.listen(source, timeout=10)
                print("Recording done, processing")
                
            guess = int(r.recognize_google(audio_text, language="en-US"))
            attempts += 1

            if guess < secret_number:
                print("Too low! Try again.")
            elif guess > secret_number:
                print("Too high! Try again.")
            else:
                print(f"Congratulations! You guessed the number {secret_number} in {attempts} attempts.")
                break
        except (ValueError, sr.UnknownValueError):
            print("Invalid input or could not recognize. Please try again.")

        display_attempts(frame, attempts, max_attempts)  # Pass 'frame' as the first argument

    if attempts >= max_attempts:
        print(f"Sorry, you've reached the maximum number of attempts. The secret number was {secret_number}.")#displaying message after running out of attempts

def display_attempts(frame, attempts, max_attempts):
    text_y_position = 70
    cv2.putText(frame, f"Attempts: {max_attempts - attempts}/{max_attempts}", (10, text_y_position + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

def main(video_capture):
    # Initialize the face cascade classifier
    cascPath = 'haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
    
    # Initialize the speech recognizer
    r = sr.Recognizer()
    applying_red_filter = False
    detect_face_start_time = None

    while True:
        # Capture a frame from the webcam
        ret, frame = video_capture.read()
        
        # Detect faces using the face cascade classifier
        faces = detect_faces(frame, faceCascade)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # Display the number of detected faces and current time
        num_faces = len(faces)
        cv2.putText(frame, f"Detected Faces: {num_faces}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cv2.putText(frame, f"Time: {current_time}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        text_y_position = 70  # Initial Y position for text
        
        # Display the instructions based on the current state
        if not applying_red_filter:
            cv2.putText(frame, "Say 'red' to Apply Red Filter", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            text_y_position += 20  # Increase Y position for the next message

        cv2.putText(frame, "Say 'detectface' for face detection", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        text_y_position += 20
        cv2.putText(frame, "Say 'Start game' to play game", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        text_y_position += 20
        cv2.putText(frame, "Say 'screenshot' to take a picture", (10, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        
        if detect_face_start_time is not None:
            elapsed_time = (datetime.datetime.now() - detect_face_start_time).total_seconds()
            if elapsed_time < 2:
                cv2.putText(frame, "Image has been Recognized", (10, text_y_position + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                detect_face_start_time = None  # Reset the start time when time limit exceeded
                cv2.destroyWindow("Detect Face")  # Close the "Detect Face" window
        
        # Display the frame with all the information
        cv2.imshow('Face Detection', frame)
        
        # Listen for voice commands using the microphone
        with sr.Microphone() as source:
            print("Please say something")
            audio_text = r.listen(source, timeout=100)
            print("Recording done, processing")

        try:
            text = r.recognize_google(audio_text, language="en-US")
            print("You said: " + text)
            
            # Check for recognized voice commands and perform corresponding actions
            if text.lower() == "detect face":
                frame = cv2.imread(image_path)  # Load the image
                recognize_uploaded_image(image_path, faceCascade, frame)
                
            elif text.lower() == "screenshot":  # Check for "screenshot" command
                screenshot_filename = f"screenshot_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.png"
                cv2.imwrite(screenshot_filename, frame)  # Save the screenshot
                print(f"Screenshot saved as {screenshot_filename}")
                
            elif text.lower() == "red":  # Check for "red" command
                red_filtered_frame = apply_red_filter(frame)  # Apply red filter
                cv2.imshow('Face Detection', red_filtered_frame)  # Update the frame with red filter
                
            elif text.lower() == "start game":
                play_number_guessing_game(r, frame)  # Start the number guessing game
                
            elif text.lower() == "stop":#check for stop command
                print("Program stopped")#when detected display program stopped
                break
                
                
        except sr.UnknownValueError:
            print("Listening...")
        except sr.RequestError:
            print("Sorry, there was an issue with the Google Speech Recognition service")
            
        # Check for user input to quit the program
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break
            
    # Release the video capture and close all windows
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Initialize the webcam video capture
    video_capture = cv2.VideoCapture(0)
    video_capture.set(3, 640) #set width
    video_capture.set(4, 480) #set height
    main(video_capture) #start main program loop
    video_capture.release() #release video capture
    cv2.destroyAllWindows() #close all windowns

Please say something
Recording done, processing
Listening...
Please say something
Recording done, processing
You said: stop
Program stopped
